In [3]:
# Load kedro environment (not needed in .py)
from pathlib import Path
from kedro.framework.context import load_context

# Load a context to be able to work in the notebook
#current_dir = Path.cwd()
current_dir = Path("/u01/share/cesar/aa_engine_uy/notebooks/")
proj_path = current_dir.parent
context = load_context(proj_path)
catalog = context.catalog
credentials = context.config_loader.get("credentials*","credentials*/**")
parameters = context.config_loader.get("parameters*","parameters*/**")

from aa_engine_pkg.assets.utils import *
from aa_engine_pkg.assets.core.data.kedro.catalog_expansion.partitioned_sql import SQLPartitionedDataSet
from datetime import datetime

In [2]:
upgrades=catalog.load('upgrades_basicos')

2021-01-08 15:07:52,048 - kedro.io.data_catalog - INFO - Loading data from `upgrades_basicos` (SQLPartitionedDataSet)...


In [50]:
def prod_moves(upgrades:SQLPartitionedDataSet,aniomes:str):
    start_date=pd.to_datetime(aniomes+"01")
    end_date = start_date + relativedelta(months=1) - timedelta(days=1)
    start_date=start_date.strftime('%Y%m%d')
    end_date=end_date.strftime('%Y%m%d')
    print(start_date, "|",end_date)
    # Load data for required period
    df_prod_change = upgrades.filter_by(date=[start_date,end_date])
    
    df_prod_change=df_prod_change.loc[df_prod_change.EVENTO_ID.isin([107,108])].sort_values(['CUSTOMER_ID','FECHA'])
    
    df_prod_change=df_prod_change.groupby('CUSTOMER_ID').head(2)
    
    old=df_prod_change.loc[df_prod_change.EVENTO_ID==107]
    new=df_prod_change.loc[df_prod_change.EVENTO_ID==108]
    movs=pd.merge(old,new,on="CUSTOMER_ID",how="left")
    
    print(f'clientes total {movs.shape[0]}')
    movs=movs[["CUSTOMER_ID","PRODUCTO_x","PRODUCTO_y"]]
    movs["perido"]=aniomes
    path=f'/u01/share/cesar/others/upsell_uy/{aniomes}'
    movs.to_parquet(path)

In [51]:
aniomes=["202001","202002","202003","202004","202005","202006","202007","202008","202009","202010","202011","202012"]

In [52]:
for x in aniomes:
    prod_moves(upgrades,x)

20200101 | 20200131
select * from stg_uy_plan_evento where PROD_CATEGORY_ID = 3 and EVENTO_ID IN (107,108,133,142) and FECHA >= to_date('20200101', 'yyyymmdd') and FECHA < to_date('20200131', 'yyyymmdd')
clientes total 1438
20200201 | 20200229
select * from stg_uy_plan_evento where PROD_CATEGORY_ID = 3 and EVENTO_ID IN (107,108,133,142) and FECHA >= to_date('20200201', 'yyyymmdd') and FECHA < to_date('20200229', 'yyyymmdd')
clientes total 1307
20200301 | 20200331
select * from stg_uy_plan_evento where PROD_CATEGORY_ID = 3 and EVENTO_ID IN (107,108,133,142) and FECHA >= to_date('20200301', 'yyyymmdd') and FECHA < to_date('20200331', 'yyyymmdd')
clientes total 1338
20200401 | 20200430
select * from stg_uy_plan_evento where PROD_CATEGORY_ID = 3 and EVENTO_ID IN (107,108,133,142) and FECHA >= to_date('20200401', 'yyyymmdd') and FECHA < to_date('20200430', 'yyyymmdd')
clientes total 1149
20200501 | 20200531
select * from stg_uy_plan_evento where PROD_CATEGORY_ID = 3 and EVENTO_ID IN (107,10

In [53]:
df=list()
for x in aniomes:
    path=f'/u01/share/cesar/others/upsell_uy/{x}'
    movs=pd.read_parquet(path)
    print(f'clientes total {movs.shape[0]}')
    df.append(movs)
M=pd.concat(df)

clientes total 1438
clientes total 1307
clientes total 1338
clientes total 1149
clientes total 1459
clientes total 1629
clientes total 1620
clientes total 1340
clientes total 1376
clientes total 1736
clientes total 1444
clientes total 1489


In [54]:
M.to_csv(f'/u01/share/cesar/others/upsell_uy/movimientos.csv')